# 잔기별 항원-항체 간 최소 거리 및 면적 기반 epitope labeling 

In [1]:
"""
8DB4 복합체를 이용해 Ara h 2 에피토프 라벨 생성 (거리 + ΔSASA Biopython SR 사용)
ASA = SASA
"""
import os
import numpy as np
import pandas as pd
from pathlib import Path
from Bio.PDB import PDBParser           # PDB 파일을 파싱해 구조(Structure) 객체 생성
from Bio.PDB.SASA import ShrakeRupley   # 용매 접근 표면적(SASA/ASA) 계산을 위한 알고리즘
from Bio.PDB import StructureBuilder    # 항원 체인만으로 monomer 구조를 메모리에 구성

# ========== 1) 실행 설정(사용자가 상황에 맞게 바꾸는 값) ==========
PDB_PATH = "D:/final/8DB4/8DB4_mono/8DB4.pdb"   # 8DB4 PDB 파일 경로
MAP_CSV  = "D:/final/8DB4/8DB4_mono/mapping_out/Q6PSU2_to_8DB4_chainE_mapping.csv"  # 매핑 CSV
OUT_CSV  = "AraH2_epitope_labels_from_8DB4.csv"

ANTIGEN_CHAINS  = {"E"}                  # 항원 체인(8DB4에서 Ara h 2: E/F 중 E를 사용)
ANTIBODY_CHAINS = {"A","B","C","D"}      # 항체 체인(8DB4에서 E에 결합: A/B, C/D)
DIST_CUTOFF  = 4.0                       # 거리 임계값(Å) — 에피토프 접촉 판정에 사용
DSASA_CUTOFF = 1.0                       # ΔSASA 임계값(Å^2) — 인터페이스 판정 보강
HEAVY_ATOMS_ONLY = True                  # 거리 계산 시 수소(H) 제외할지 여부

### PDB 잔기 식별자
PDB(Protein Data Bank) 파일 형식에서 각 잔기(residue)는 체인 ID, 잔기 서열 번호, 그리고 삽입 코드의 조합으로 고유하게 식별.

1. chain_id (chain): 잔기가 속한 거대분자 사슬(macromolecular chain)을 식별. PDB 구조는 여러 개의 사슬을 포함(예: 항체에 대한 'A', 'B', 'C', 'D' 사슬과 항원에 대한 'E' 사슬). res.get_parent().id 메서드가 이 식별자를 가져옴.

2. hetflag: HETATM 플래그. 이는 잔기가 물(HOH), 리간드, 또는 이온과 같은 비표준 잔기("hetero" atom)인지를 나타내는 문자열. 이 플래그가 비어 있지 않은 문자열(예: 'W')로 존재하면 해당 잔기는 HETATM이며, 표준 아미노산이나 뉴클레오타이드가 아님. 코드에서는 hetflag.strip()을 통해 비어 있지 않은 문자열인지 확인하고, HETATM은 스킵.

3. resseq: 잔기 서열 번호로, 잔기가 해당 체인 내에서 차지하는 위치를 나타내는 정수. 예를 들어, resseq=150인 잔기는 해당 체인의 150번째 잔기.

4. icode: 삽입 코드(insertion code)의 약어로, 동일한 잔기 서열 번호를 가진 잔기들을 구별하기 위해 사용되는 한 글자(주로 문자)의 문자열. 삽입 코드는 일반적으로 폴리펩타이드 사슬이 원래 서열과 다른 수의 잔기를 가질 때 사용되며, 이는 역사적인 번호 매기기 시스템이나 삽입으로 인한 경우가 많음. 잔기에 삽입 코드가 없는 경우, 이 값은 빈 문자열 또는 None. 코드 (icode or "").strip()는 이 값을 None이거나 공백만 포함하는 경우 간단한 빈 문자열로 정규화.

In [2]:
# ========== 2) 유틸리티 함수들 ==========
def residue_key_from_biopdb(res):
    """
    Bio.PDB Residue 객체에서 (chain, resseq, icode) 키를 추출.
    - HETATM(물/리간드 등)은 제외.
    - insertion code(icode)가 None/' '이면 빈 문자열로 정규화.
    """
    chain_id = res.get_parent().id
    hetflag, resseq, icode = res.id
    if hetflag.strip():  # HETATM은 스킵
        return None
    icode = (icode or "").strip()
    return (chain_id, int(resseq), icode)

In [3]:
def get_atom_coord_array(res, heavy_only=True):
    """
    잔기의 원자 좌표를 (N,3) numpy 배열로 반환.
    - heavy_only=True이면 수소(H) 원자 제외.
    """
    coords = []                                     # 좌표를 담을 리스트
    for a in res.get_atoms():                       # 잔기 내 모든 원자 순회
        if heavy_only and a.element == "H":         # 수소 제외 옵션이면 건너뜀
            continue
        coords.append(a.coord)                      # 3차원 좌표(np.array([x,y,z]))
    if not coords:                                  
        return np.zeros((0,3), dtype=float)         # 원자가 하나도 없으면 빈 (0,3) 배열 반환
    return np.vstack(coords)                        # 리스트를 (N,3) 배열로 변환해 반환

In [4]:
def sr_residue_sasa_from_structure(structure, chains):
    """
    Shrake–Rupley 알고리즘으로 구조 전체의 per-residue ASA 계산 후,
    지정한 체인 집합(chains)에 속한 잔기만 딕셔너리로 추출.
    - 반환: {(chain, resseq, icode): ASA_float}
    """
    sr = ShrakeRupley()                             # 기본: probe 1.4Å, 점수 100
    sr.compute(structure, level="R")                # per-residue 수준으로 ASA 계산
    areas = {}                                      
    for ch in structure.get_chains():              
        if ch.id not in chains:                     # 모든 체인에 대해 관심 체인만 필터링
            continue
        for res in ch.get_residues():               # 체인의 모든 잔기 순회
            het, resseq, icode = res.id
            if het.strip():                         # HETATM(리간드/물 등)은 제외
                continue
            icode = (icode or "").strip()           # icode 정규화
            asa = getattr(res, "sasa", None)        # SR가 계산해서 잔기에 기록한 ASA 값
            if asa is not None:                     
                areas[(ch.id, int(resseq), icode)] = float(asa)  # 값이 존재하면 딕셔너리에 저장
    return areas                                    # per-residue ASA 딕셔너리 반환

In [5]:
def build_antigen_monomer_structure(structure, antigen_chains):
    """
    전체 구조에서 항원 체인만 골라 새 구조를 메모리상에 구성.
    - 파일로 쓰지 않고 계산할 수 있도록 StructureBuilder 사용.
    """
    sb = StructureBuilder.StructureBuilder()        
    sb.init_structure("mono")                       # 구조 ID 설정
    sb.init_model(0)                                # 모델 인덱스 0 초기화
    for ch in structure.get_chains():               # 원본 구조의 체인 순회
        if ch.id not in antigen_chains:             # 항원 체인만 복사
            continue
        sb.init_chain(ch.id)                        # 새 구조에 동일 체인 ID 생성
        for res in ch.get_residues():               # 체인 내 잔기 순회
            het, resseq, icode = res.id
            if het.strip():                         # HETATM 제외
                continue
            sb.init_seg(" ")                         # 세그먼트 ID(필요 시 공백)
            sb.init_residue(res.get_resname(),       # 잔기 3-letter 코드(e.g., 'ALA')
                             het,                    # HET 플래그(' ' 정상 잔기)
                             resseq,                 # 연속 번호
                             icode or " ")           # 삽입 코드(공백 허용)
            for atom in res.get_atoms():             # 잔기의 모든 원자 복사
                sb.init_atom(atom.id,                # 원자 이름(e.g., 'CA')
                             atom.coord,             # 좌표
                             atom.bfactor,           # B-factor
                             atom.occupancy,         # 점유율
                             atom.altloc,            # 대체 위치 코드
                             atom.fullname,          # 전체 원자명
                             atom.element)           # 원소 기호
    return sb.get_structure()                        # 새로 만든 항원 단독 구조

In [6]:
# ========== 3) PDB 로드 & 항원/항체 원자·잔기 수집 ==========
# - 8DB4 파일을 파싱하여 구조 객체 생성
# - 항원 체인에 속한 잔기 리스트, 항체(파트너) 모든 원자 좌표 배열 생성

parser = PDBParser(QUIET=True)
structure = parser.get_structure("x", PDB_PATH)

antigen_residues = []  # 항원 잔기 리스트
antibody_atoms   = []  # 항체 원자 리스트
for chain in structure.get_chains():
    cid = chain.id
    for res in chain.get_residues():
        key = residue_key_from_biopdb(res) # (체인, resseq, icode) 키 추출(유효 잔기만)
        if key is None:                    # HETATM 등은 None이므로 스킵
            continue
        if cid in ANTIGEN_CHAINS:
            antigen_residues.append(res)   # 항원 체인이라면 항원 잔기 목록에 추가
        elif cid in ANTIBODY_CHAINS:
            for a in res.get_atoms():      # 항체 체인이라면 잔기 모든 원자 순회 - 항체 원자들을 모아 나중에 거리 계산에 사용
                if HEAVY_ATOMS_ONLY and a.element == "H":
                    continue
                antibody_atoms.append(a)   # 항체 원자 목록에 추가

# 항체 원자 좌표를 (N,3) 배열로 구성 (없으면 (0,3) 크기의 빈 배열)
AB = np.vstack([a.coord for a in antibody_atoms]) if len(antibody_atoms) else np.zeros((0,3))
print(f"[INFO] Antigen residues: {len(antigen_residues)}, Antibody atoms: {len(AB)}")

[INFO] Antigen residues: 95, Antibody atoms: 6346


In [7]:
# ========== 4) 항원 잔기별 최소 거리(min_dist_to_partner) 계산 ==========
# - 각 항원 잔기의 heavy-atom 집합과 항체 원자들의 최소 거리를 계산

pdb_rows = []                                       # 각 항원 잔기에 대한 계산 결과를 저장할 리스트
for res in antigen_residues:
    key = residue_key_from_biopdb(res)
    if key is None:
        continue
    chain_id, resseq, icode = key
    ag = get_atom_coord_array(res, heavy_only=HEAVY_ATOMS_ONLY)  # 항원 잔기의 원자좌표 배열
    if len(ag) == 0 or len(AB) == 0:
        dmin = np.nan                               # 항원/항체 좌표가 없으면 최소 거리 계산 불가 → NaN
    else:
        # 모든 쌍 거리 계산: (n_ag,1,3) - (1,n_ab,3) → (n_ag, n_ab, 3) → L2 norm → 최솟값
        diff = ag[:,None,:] - AB[None,:,:]
        dmin = float(np.min(np.linalg.norm(diff, axis=2)))
    pdb_rows.append({
        "pdb_chain": chain_id,                      # 체인 ID
        "pdb_resseq": int(resseq),                  # 잔기 번호
        "pdb_icode": (icode or "").strip(),         # 삽입 코드(공백 정규화)
        "min_dist_to_partner": dmin,                # 항체와의 최소 heavy-atom 거리
    })

df_dist = pd.DataFrame(pdb_rows)

In [8]:
# ========== 5) per-residue SASA(=ASA) (Shrake–Rupley로 complex/monomer 계산) ==========
# - complex: 전체 구조에서 항원 체인 잔기의 ASA만 추출
# - monomer: 항원 체인만 복사한 구조를 만들어 ASA 계산

import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
warnings.simplefilter("ignore", PDBConstructionWarning)

sasa_complex = sr_residue_sasa_from_structure(structure, ANTIGEN_CHAINS)  # 복합체에서 항원 잔기 ASA
mono_struct  = build_antigen_monomer_structure(structure, ANTIGEN_CHAINS) # 항원만 복사한 monomer 구조
sasa_monomer = sr_residue_sasa_from_structure(mono_struct, ANTIGEN_CHAINS) # monomer에서 항원 잔기 ASA

# ΔSASA = monomer - complex (항원 체인 잔기 대상) > 항원 잔기가 복합체 결합으로 얼마나 가려졌는지
for rec in pdb_rows:
    key = (rec["pdb_chain"], rec["pdb_resseq"], (rec["pdb_icode"] or "").strip())
    key_noicode = (key[0], key[1], "")  # RSA와 달리 SR은 icode를 유지하지만, 안전하게 공백 키도 확인
    area_c = sasa_complex.get(key, sasa_complex.get(key_noicode, np.nan))  # 복합체 ASA 조회
    area_m = sasa_monomer.get(key, sasa_monomer.get(key_noicode, np.nan))  # monomer ASA 조회
    rec["sasa_complex"] = float(area_c) if area_c is not None else np.nan  # 복합체 ASA 기록
    rec["sasa_monomer"] = float(area_m) if area_m is not None else np.nan  # monomer ASA 기록
    rec["delta_sasa"] = (rec["sasa_monomer"] - rec["sasa_complex"]         # ΔSASA 계산
                         if (not np.isnan(rec["sasa_monomer"]) and not np.isnan(rec["sasa_complex"]))
                         else np.nan)

df_sasa = pd.DataFrame(pdb_rows)

# 거리(df_dist)와 ΔSASA(df_sasa)를 키(체인,번호,icode)로 병합 → 항원 PDB 테이블
df_pdb = df_dist.merge(
    df_sasa[["pdb_chain","pdb_resseq","pdb_icode","sasa_complex","sasa_monomer","delta_sasa"]],
    on=["pdb_chain","pdb_resseq","pdb_icode"],
    how="outer"
)

In [9]:
# ========== 6) 매핑 CSV와 병합 ==========
# - 키: (pdb_chain, pdb_resseq, pdb_icode)
# - 결과: UniProt 좌표계(성숙체 기준 idx_mature 포함)와 PDB 기반 측정치(거리/ΔSASA)를 한 테이블에 통합

df_map = pd.read_csv(MAP_CSV)
df_map["pdb_resseq"] = pd.to_numeric(df_map["pdb_resseq"], errors="coerce").astype("Int64")
df_map["pdb_icode"]  = df_map["pdb_icode"].fillna("").astype(str).str.strip()
df_pdb["pdb_resseq"] = pd.to_numeric(df_pdb["pdb_resseq"], errors="coerce").astype("Int64")
df_pdb["pdb_icode"]  = df_pdb["pdb_icode"].fillna("").astype(str).str.strip()

key = ["pdb_chain","pdb_resseq","pdb_icode"]
df = df_map.merge(df_pdb, on=key, how="left") # 매핑과 PDB 측정치 결합(왼쪽 기준: 매핑 유지)

In [10]:
# ========== 7) 라벨 계산 (거리 / ΔSASA / 최종) ==========
# - label_dist : 최소 거리 ≤ 4.0 Å → 1, 아니면 0
# - label_sasa : ΔSASA > 1.0 Å² → 1, 아니면 0
# - label_epitope : 두 조건 AND (보수적; 필요시 OR로 바꿔 민감도↑)

df["label_dist"] = (df["min_dist_to_partner"] <= DIST_CUTOFF).astype("Int64")
df["label_sasa"] = (df["delta_sasa"] > DSASA_CUTOFF).astype("Int64")
df["label_epitope"] = (df["label_dist"].eq(1) & df["label_sasa"].eq(1)).astype("Int64")

# 편향 방지 마스킹:
# - 성숙체 Ara h 2에 속하지 않는 aa / 구조·면적 결측 → label_epitope = NaN
if "is_mature" in df.columns:
    df.loc[~df["is_mature"], "label_epitope"] = np.nan   # 성숙체 Ara h 2에 속하지 않는 aa → 라벨 NA
df.loc[df["min_dist_to_partner"].isna(), "label_epitope"] = np.nan   # 거리 계산 불가 → NA
df.loc[df["delta_sasa"].isna(), "label_epitope"] = np.nan            # ΔSASA 계산 불가 → NA

In [11]:
# ========== 8) 저장 ==========
# - 최종 결과 CSV로 저장 (매핑 + 거리 + ΔSASA + 라벨)

df.to_csv(OUT_CSV, index=False)
print(f"[DONE] Saved -> {OUT_CSV}")

[DONE] Saved -> AraH2_epitope_labels_from_8DB4.csv


In [12]:
# ========== 9) 요약 출력 ==========
# - 전체 행수, 거리/ΔSASA=1인 residue 개수, 최종 에피토프 양성 개수 등 간단 통계

summary = {
    "rows_total": len(df),
    "antigen_residues_measured": int(df["min_dist_to_partner"].notna().sum()),
    "label_dist_pos": int(df["label_dist"].fillna(0).eq(1).sum()),
    "label_sasa_pos": int(df["label_sasa"].fillna(0).eq(1).sum()),
    "label_epitope_pos": int(pd.to_numeric(df["label_epitope"], errors="coerce").fillna(0).eq(1).sum()),
}
print(summary)

{'rows_total': 172, 'antigen_residues_measured': 95, 'label_dist_pos': 28, 'label_sasa_pos': 55, 'label_epitope_pos': 28}
